<h1><span style="color: #6495ED;">Elevating Air Quality Predictions: Unleashing XGBoost for Accurate Air Quality Index Modeling</span></h1>

Prepared by Lipsita Tripathy

Email: lipsitalt@gmail.com

January 2024

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go
import xgboost as xgb
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('../../data/generated/Delhi_AQI_final_df_before_modeling.csv')
df.head()

,Datetime,AQI,PM2.5 (ug/m3),PM10 (ug/m3),NOx (ug/m3),NH3 (ug/m3),SO2 (ug/m3),CO (ug/m3),Ozone (ug/m3),RH (%),...,t_CO (ug/m3),t_Ozone (ug/m3),t_WS (m/s),t_SR (W/mt2),t_Volatility_Last_24hr,t_Volatility_Last_7d,t_Volatility_Last_30d,t_AQI_lag_24hr,t_AQI_lag_48hr,t_AQI_lag_168hr
0,2013-01-01 00:00:00,354.0,290.774583,292.631667,117.224563,75.685556,9.992130,5.052250,13.794630,88.716778,...,1.800430,2.694264,0.314162,3.714195,2.230820,3.405326,4.138828,5.831882,5.961005,6.075346
1,2013-01-01 01:00:00,358.0,275.749821,296.150000,88.122976,66.740556,9.477546,7.665310,9.977963,89.612778,...,2.159328,2.395890,0.330103,3.673794,2.353812,3.397713,4.138876,5.823046,5.966147,6.073045
2,2013-01-01 02:00:00,362.0,271.463472,309.030000,61.464690,57.030556,9.207963,10.777421,7.316574,91.010556,...,2.466184,2.118250,0.329304,3.455326,2.475083,3.389909,4.138987,5.814131,5.971262,6.068426
3,2013-01-01 03:00:00,367.0,279.071667,317.826667,47.583524,43.298333,10.871667,11.793810,7.910146,91.930000,...,2.548961,2.187191,0.326422,3.079282,2.592957,3.382300,4.139163,5.805135,5.973810,6.061457
4,2013-01-01 04:00:00,370.0,269.118333,308.521667,43.535333,32.023333,11.020833,10.027778,9.348849,92.335556,...,2.400417,2.336875,0.379197,2.525195,2.692013,3.374865,4.139385,5.802118,5.971262,6.056784


In [3]:
# Convert 'Datetime' column to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])

df = df.set_index('Datetime')

In [4]:
original_order = ['PM2.5 (ug/m3)','PM10 (ug/m3)','NOx (ug/m3)','NH3 (ug/m3)','SO2 (ug/m3)','CO (ug/m3)','Ozone (ug/m3)','RH (%)','WS (m/s)','WD (degree)','BP (mmHg)','AT (degree C)','RF (mm)','SR (W/mt2)']

# Use reindex to maintain the original order
selected_columns = df.loc[:, original_order]

# Define date ranges
date_ranges = [
    ('2013-01-01', '2022-03-01'),  # For training set
    ('2022-03-01', '2023-03-30')   # For test set
]

# Create DMatrices for training and testing
dmatrices = [
    (
        xgb.DMatrix(selected_columns.loc[(df.index >= start) & (df.index < end)], label=df.loc[(df.index >= start) & (df.index < end), 'y_AQI']),
        df.loc[(df.index >= start) & (df.index < end), 'y_AQI']
    )
    for start, end in date_ranges
]

dtrain, y_train = dmatrices[0]
dtest, y_test = dmatrices[1]

In [5]:
# Get the ordered feature list from dtest
feature_names = dtrain.feature_names

# Display the feature list and verify that the features are in order
print(feature_names)

['PM2.5 (ug/m3)', 'PM10 (ug/m3)', 'NOx (ug/m3)', 'NH3 (ug/m3)', 'SO2 (ug/m3)', 'CO (ug/m3)', 'Ozone (ug/m3)', 'RH (%)', 'WS (m/s)', 'WD (degree)', 'BP (mmHg)', 'AT (degree C)', 'RF (mm)', 'SR (W/mt2)']


In [6]:
X_train = dtrain.get_data()
X_test = dtest.get_data()

In [7]:
# Set default hyperparameters
params = {
    'objective': 'reg:squarederror',  # Regression task
    'eval_metric': 'rmse',            # Evaluation metric: Root Mean Squared Error
}

# Train the baseline model
num_round = 100  # You can adjust the number of boosting rounds
baseline_model = xgb.train(params, xgb.DMatrix(X_train, label=y_train), num_round, 
                           evals=[(xgb.DMatrix(X_test, label=y_test), 'test')],
                           early_stopping_rounds=10)

# Make predictions on the test set
y_test_pred = baseline_model.predict(xgb.DMatrix(X_test))


[0]	test-rmse:85.67183
[1]	test-rmse:70.86995
[2]	test-rmse:62.20282
[3]	test-rmse:57.45371
[4]	test-rmse:54.64125
[5]	test-rmse:53.08375
[6]	test-rmse:52.24833


[7]	test-rmse:52.01521
[8]	test-rmse:51.71562
[9]	test-rmse:51.69916
[10]	test-rmse:51.51106
[11]	test-rmse:51.54093
[12]	test-rmse:51.46294
[13]	test-rmse:51.71147
[14]	test-rmse:51.70851
[15]	test-rmse:51.79939
[16]	test-rmse:51.80028
[17]	test-rmse:51.82408
[18]	test-rmse:51.84122
[19]	test-rmse:51.78534
[20]	test-rmse:51.85860
[21]	test-rmse:51.89203
[22]	test-rmse:51.84983


In [8]:
from sklearn.metrics import r2_score

# Make predictions on the scaled test set
y_test_pred = baseline_model.predict(xgb.DMatrix(X_test))

# Evaluate the model on the scaled test set
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"Baseline Model RMSE on Test Set: {rmse_test}")

# Calculate R-squared
r2_test = r2_score(y_test, y_test_pred)
print(f"Baseline Model R-squared on Test Set: {r2_test}")

Baseline Model RMSE on Test Set: 51.849830696217964
Baseline Model R-squared on Test Set: 0.7585535667085506


In [9]:
import pickle

# Save the XGBoost model to a .pkl file using pickle
model_path_pkl = "xgb_aqi_model____no_rolling_window_data.pkl"
with open(model_path_pkl, "wb") as file:
    pickle.dump(baseline_model, file)

In [10]:

df_slim_copy = df[original_order + ['y_AQI']].copy().sample(n=5000)

# df_slim_copy.to_csv("sample_test.csv", index=False)

df_slim_copy.head()

,PM2.5 (ug/m3),PM10 (ug/m3),NOx (ug/m3),NH3 (ug/m3),SO2 (ug/m3),CO (ug/m3),Ozone (ug/m3),RH (%),WS (m/s),WD (degree),BP (mmHg),AT (degree C),RF (mm),SR (W/mt2),y_AQI
Datetime,,,,,,,,,,,,,,,
2022-09-26 08:00:00,59.221390,118.041092,95.553333,25.695842,7.325361,0.870000,31.723847,82.320869,0.549897,160.048153,972.555606,26.009048,0.000000,87.117830,106.0
2020-10-19 01:00:00,132.165810,264.509714,70.415000,38.165632,17.758556,1.546667,10.641728,49.318621,0.710741,230.123333,953.593467,23.275000,0.001111,10.558276,265.0
2019-04-18 11:00:00,37.491802,92.881010,19.490000,26.878095,16.990739,0.768333,62.105741,36.728214,1.020128,201.551373,960.473733,27.755606,0.000000,547.125000,136.0
2022-06-29 13:00:00,66.031286,253.211176,14.246667,54.890140,9.988436,1.436667,65.234364,47.785372,1.754946,139.268846,965.382267,37.918986,0.000000,456.436307,120.0
2015-05-26 10:00:00,65.300000,372.765000,69.238750,40.203333,9.138452,2.921429,39.346190,28.944762,1.463333,81.786667,853.055714,38.599167,0.000000,316.676667,272.0


### Verify Prediction by loading the saved Model

In [11]:
import joblib

features_for_prediction = df_slim_copy.drop(columns=['y_AQI'])

# Convert the Pandas DataFrame to a DMatrix
features_for_prediction_dmatrix = xgb.DMatrix(features_for_prediction)

# Load the XGBoost model
model_path = "xgb_aqi_model____no_rolling_window_data.pkl"
xgb_model = joblib.load(model_path)

# Make predictions using the DMatrix
predictions = xgb_model.predict(features_for_prediction_dmatrix)

df_slim_copy['pred_val'] = predictions

df_slim_copy.head()

,PM2.5 (ug/m3),PM10 (ug/m3),NOx (ug/m3),NH3 (ug/m3),SO2 (ug/m3),CO (ug/m3),Ozone (ug/m3),RH (%),WS (m/s),WD (degree),BP (mmHg),AT (degree C),RF (mm),SR (W/mt2),y_AQI,pred_val
Datetime,,,,,,,,,,,,,,,,
2022-09-26 08:00:00,59.221390,118.041092,95.553333,25.695842,7.325361,0.870000,31.723847,82.320869,0.549897,160.048153,972.555606,26.009048,0.000000,87.117830,106.0,149.770859
2020-10-19 01:00:00,132.165810,264.509714,70.415000,38.165632,17.758556,1.546667,10.641728,49.318621,0.710741,230.123333,953.593467,23.275000,0.001111,10.558276,265.0,277.861053
2019-04-18 11:00:00,37.491802,92.881010,19.490000,26.878095,16.990739,0.768333,62.105741,36.728214,1.020128,201.551373,960.473733,27.755606,0.000000,547.125000,136.0,152.377213
2022-06-29 13:00:00,66.031286,253.211176,14.246667,54.890140,9.988436,1.436667,65.234364,47.785372,1.754946,139.268846,965.382267,37.918986,0.000000,456.436307,120.0,138.080597
2015-05-26 10:00:00,65.300000,372.765000,69.238750,40.203333,9.138452,2.921429,39.346190,28.944762,1.463333,81.786667,853.055714,38.599167,0.000000,316.676667,272.0,263.237732


In [12]:
# # df_slim_copy_2 = df_slim_copy.copy().sample(n=1000)
# df_slim_copy_2 = df_slim_copy[(df_slim_copy['pred_val'] - df_slim_copy['y_AQI']).between(-30, 30)].sample(5000)

# df_slim_copy_2.drop(columns=['pred_val'], inplace=True)  # Drop the 'pred_val' column
# df_slim_copy_2.to_csv("sample_test.csv", index=False)

# df_slim_copy_2.head()